In [1]:
import pandas as pd
import numpy as np
import pickle
import gc
import os
import time
import copy
import multiprocessing as mp
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from scipy import sparse, spatial
import warnings
warnings.filterwarnings('ignore')
tqdm.pandas()

In [2]:
tic = time.time()
DATA_PATH = '../pkl/'
SAVE_PATH = './feats/'
if not os.path.exists(SAVE_PATH):
    print('create dir: %s' % SAVE_PATH)
    os.mkdir(SAVE_PATH)

print('gen_prev_ans_feat...')

gen_prev_ans_feat...


In [3]:
inv_last_answer_time = pd.read_pickle(os.path.join(DATA_PATH, 'inv_last_answer_time.pkl'))
answer_info = pd.read_pickle(os.path.join(DATA_PATH, 'answer_info_tiny.pkl'))

In [4]:
answer_info['user_time'] = answer_info['uid'].astype(str) + '_' + answer_info['atime'].astype(str)
answer_info.sort_values(by=['uid', 'user_time'],inplace=True)

In [6]:
answer_info['cnt'] = 1
cols = ['excellent', 'recommend', 'figure', 'video', 'num_word', 'num_like', 'num_unlike', 'num_comment', 'num_favor', 'num_thank', 'num_report', 'num_nohelp', 'num_oppose', 'cnt']
prev_ans_stat = answer_info.groupby('uid')[cols].cumsum()
prev_ans_stat.columns = [('prev_' + c + '_sum') for c in cols]
mean_arr = prev_ans_stat.values[:, :-1] / prev_ans_stat.values[:, -1].reshape((-1, 1))
prev_ans_stat = pd.concat([prev_ans_stat, pd.DataFrame(mean_arr, columns=[('prev_' + c + '_mean') for c in cols[:-1]])], axis=1)
prev_ans_stat['user_time'] = answer_info['user_time']
prev_ans_stat.drop_duplicates(subset='user_time', keep='last', inplace=True)

In [7]:
tmp = inv_last_answer_time.merge(prev_ans_stat, 'left', 'user_time')
tmp = tmp[[('prev_' + c + '_sum') for c in cols] + [('prev_' + c + '_mean') for c in cols[:-1]]].reset_index(drop=True)
tmp.head(2)

,prev_excellent_sum,prev_recommend_sum,prev_figure_sum,prev_video_sum,prev_num_word_sum,prev_num_like_sum,prev_num_unlike_sum,prev_num_comment_sum,prev_num_favor_sum,prev_num_thank_sum,...,prev_video_mean,prev_num_word_mean,prev_num_like_mean,prev_num_unlike_mean,prev_num_comment_mean,prev_num_favor_mean,prev_num_thank_mean,prev_num_report_mean,prev_num_nohelp_mean,prev_num_oppose_mean
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
tmp.to_pickle(os.path.join(SAVE_PATH, 'prev_ans_feat.pkl'))

In [9]:
toc = time.time()
print('Used time: %d' % int(toc-tic))

Used time: 147
